In [61]:
import time
import datetime
import sqlite3
import pandas as pd
import matplotlib as plt
import seaborn as sns  # für das Styling der Ausgabe https://pandas.pydata.org/pandas-docs/stable/style.html#Builtin-Styles

# Erstmal das heutige Datum feststellen
today = datetime.datetime.today()
weeknr = today.isocalendar()[1]

# Daten aus Datenbank laden
connection = sqlite3.connect('../db.sqlite3')
df = pd.read_sql_query("""SELECT bayern3.id, datum_zeit, interpret, titel, label FROM bayern3 
                        INNER JOIN songs ON bayern3.song=songs.id""", connection, index_col='id')
connection.close()

# Spalte mit der Kalenderwoche hinzufügen
df['woche'] = pd.to_datetime(df['datum_zeit']).dt.week

# ich glaube, es gibt elegantere Lösungen hierfür... https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.min.html
erstewoche = min(df.loc[:,['woche']].values.min(axis=1))

# Separaten DataFrame für den Vergleich
vergleich = pd.DataFrame()
vergleichlabel = pd.DataFrame()
# Eine Spalte je Kalenderwoche, titel als Index

for i in range(erstewoche+1, weeknr+1):
    vergleich['KW'+str(i)] = df[df['woche'] == i].groupby('titel').count().sort_values(by='datum_zeit', ascending=False)['datum_zeit']
    vergleichlabel['KW'+str(i)] = df[df['woche'] == i].groupby('label').count().sort_values(by='datum_zeit', ascending=False)['datum_zeit']

    
# Alle Zeilen droppen, die NaN enthalten (liegt vor, wenn der Titel in einer Woche nicht gespielt wurde)
vergleich = vergleich.dropna(how='any')
vergleichlabel = vergleichlabel.dropna(how='any')
# Alternativ mit 0 ersetzen
# vergleich = vergleich.fillna(value=0)

# eine Zeile löschen, z.B. NaN im Label
vergleichlabel = vergleichlabel.drop(["NaN"])
####################################
### funktionierende Auswertungen ###
####################################

tabelle = vergleichlabel.head(10)
#dfweek = pd.DataFrame(vergleich, columns=["KW12"]).sort_values(by='KW12', ascending=False)
#dftitel = pd.DataFrame(vergleich, index=["Crazy"])
#dftitel
#dfweek
cm = sns.light_palette("green", as_cmap=True)
ergebnis = tabelle.style.background_gradient(cmap=cm)
ergebnis

,KW5,KW6,KW7,KW8,KW9,KW10,KW11,KW12,KW13
label,,,,,,,,,
Interscope Records,113,109,102,107,111,91,90,92,21
Atlantic,97,97,98,104,99,103,108,106,27
RCA,97,98,83,95,84,78,82,118,24
Columbia,97,113,98,105,106,117,103,96,29
Warner Bros. Records,58,72,80,109,87,96,87,92,22
Vertigo Berlin,52,45,30,31,34,31,30,29,11
Epic,48,53,46,46,53,56,70,70,14
Island,44,46,23,29,21,26,26,30,9
Polydor,43,47,46,43,43,43,50,46,14


In [79]:
songsinlabel = pd.DataFrame()
labels = ["Interscope Records","Atlantic","RCA","Columbia","Warner Bros. Records"]

i=4
songsinlabel[labels[i]]=df[df['label'] == labels[i]].groupby('titel').count().sort_values(by='datum_zeit', ascending=False)['datum_zeit']
songsinlabel[:10]
#vergleich.T.iloc[:,0:10].plot()
#plt.pyplot.grid(True)

,Warner Bros. Records
titel,
More Than Friends,224
Perfect,115
IDGAF,115
Tip Toe,106
Heavy,35
New rules,24
Want to want me,14
Final Masquerade,10
Colors,8


In [91]:
# Einzelne Songs oder Label als Histogram über den Tag verteilt
dflabelhist = pd.DataFrame(df, columns=['datum_zeit','titel','label'])
dflabelhist['stunde'] = pd.to_datetime(dflabelhist['datum_zeit']).dt.hour
dflabelhist #= pd.DataFrame.hist(dflabelhist, bins=24)

,datum_zeit,titel,label,stunde
id,,,,
1,2018-01-27T00:05,There's Nothing Holdin' Me Back (NOTD Remix),None,0
2,2018-01-27T00:09,Échame la Culpa,None,0
3,2018-01-27T00:11,I Like Me Better,NaN,0
4,2018-01-27T00:15,Strangers,Island,0
5,2018-01-27T00:20,She Doesn't Mind,NaN,0
6,2018-01-27T00:23,Silence (DJ Damian Remix),RCA,0
7,2018-01-27T00:27,This is me,Atlantic,0
8,2018-01-27T00:32,Friends,Republic Records,0
9,2018-01-27T00:35,You know you like it,Universal Music Division Mercury Records,0
